In [2]:
from collections import defaultdict
import random
import base64
from PIL import Image
from io import BytesIO
import torch
import json

In [ ]:
USE_CUSTOM_DATASET = True
CUSTOM_DATASET_PATH = "/home/jupyter/datasphere/project/data/test.jsonl"

NUM_SAMPLES_TO_EVALUATE = 100

BASE_MODEL_DIR = "unsloth/Qwen2.5-VL-32B-Instruct-unsloth-bnb-4bit"

ADAPTER_DIR = "/home/jupyter/datasphere/project/qwen2.5-vl-32b-qlora-a100-copy"

# Имя файла для сохранения результатов остаётся прежним
RESULTS_FILENAME = "evaluation_results_ft.json"

In [4]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

if torch.cuda.is_available():
    torch.cuda.empty_cache()

PyTorch version: 2.8.0+cu128
CUDA available: True
CUDA version: 12.8
GPU: NVIDIA A100-SXM4-80GB
GPU memory: 79.3 GB


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    torch.cuda.empty_cache()

import sys
import types

def safe_import_unsloth():
    try:
        from unsloth import FastVisionModel
        print("Unsloth импортирован")
        return FastVisionModel
    except Exception as e:
        print(f"Ошибка импорта: {e}")
        # Создаем мок для обхода
        mock_utils = types.ModuleType('unsloth.kernels.utils')
        mock_utils._CUDA_STREAMS = {}
        sys.modules['unsloth.kernels.utils'] = mock_utils
        
        from unsloth import FastVisionModel
        print("Unsloth импортирован с обходом CUDA streams")
        return FastVisionModel

FastVisionModel = safe_import_unsloth()

PyTorch version: 2.8.0+cu128
CUDA available: True
GPU: NVIDIA A100-SXM4-80GB
GPU memory: 79.3 GB
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-10-15 19:27:50.196599: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-15 19:27:53.667513: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO 10-15 19:28:17 [__init__.py:216] Automatically detected platform cuda.
Switching to PyTorch attention since your Xformers is broken.

Requires Flash-Attention version >=2.7.1,<=2.8.2 but got 2.8.3.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth импортирован


In [6]:
from PIL import Image
import requests
from io import BytesIO
import json
import re
import math
import numpy as np
import datasets

In [ ]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

print("Загрузка модели Qwen2.5-VL-32B")

model, tokenizer = FastVisionModel.from_pretrained(
    model_name="unsloth/Qwen2.5-VL-32B-Instruct-unsloth-bnb-4bit",
    max_seq_length=2048,
    load_in_4bit=True,
    fast_inference=False,
    device_map="auto",
)

FastVisionModel.for_inference(model)
print(f"\nПрименение весов адаптера из локальной папки: {ADAPTER_DIR}")
model.load_adapter(ADAPTER_DIR)
FastVisionModel.for_inference(model)

print("Модель загружена")

def print_gpu_memory():
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        print(f"GPU память - Используется: {allocated:.2f} GB, Зарезервировано: {reserved:.2f} GB")

print_gpu_memory()

Загрузка модели Qwen2.5-VL-32B
==((====))==  Unsloth 2025.10.1: Fast Qwen2_5_Vl patching. Transformers: 4.56.2. vLLM: 0.11.0.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.325 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 5/5 [05:28<00:00, 65.72s/it]
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 2cf2f3ee-e077-4bee-9e88-8eb7c0004282)')' thrown while requesting HEAD https://huggingface.co/unsloth/Qwen2.5-VL-32B-Instruct-unsloth-bnb-4bit/resolve/main/generation_config.json
Retrying in 1s [Retry 1/5].
The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.



Применение весов адаптера из локальной папки: /home/jupyter/datasphere/project/qwen2.5-vl-32b-qlora-a100-copy


/home/jupyter/.local/lib/python3.10/site-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)


Модель загружена
GPU память - Используется: 23.29 GB, Зарезервировано: 30.96 GB


In [8]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:232: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  language = self._parent_header['metadata'].get('language')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:237: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  livy = self._parent_header['metadata'].get('livy')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:280: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  header = self._parent_header['header']


In [9]:
def create_stratified_subset(full_dataset, subset_size):    
    by_source = defaultdict(list)
    is_custom = 'inputs' not in full_dataset[0]

    for item in full_dataset:
        if is_custom:
            source = item.get('source', 'custom_unknown')
        else:
            source = item.get('meta', {}).get('source', 'ruCLEVR_unknown')
        by_source[source].append(item)

    total_size = len(full_dataset)
    subset = []
    
    for source, items in by_source.items():
        proportion = len(items) / total_size
        num_to_take = round(proportion * subset_size)
        
        random.shuffle(items)
        subset.extend(items[:int(num_to_take)])
        print(f"  - {source}: {len(items)} примеров ({proportion:.2%}), будет взято ~{int(num_to_take)}")

    while len(subset) < subset_size:
        source_to_add_from = max(by_source, key=lambda s: len(by_source[s]))
        for item in by_source[source_to_add_from]:
            if item not in subset:
                subset.append(item)
                break
    
    final_subset = subset[:subset_size]
    random.shuffle(final_subset)
    
    print(f"\nИтоговый размер подвыборки: {len(final_subset)}")
    return final_subset

def load_custom_jsonl_dataset(file_path):
    print(f"Загрузка кастомного датасета из {file_path}...")
    try:
        dataset = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data = json.loads(line)
                if 'images' in data and data['images']:
                    data['images'] = [base64.b64decode(img_str) for img_str in data['images']]
                dataset.append(data)
        print(f"Датасет загружен: {len(dataset)} примеров.")
        return dataset
    except FileNotFoundError:
        print(f"ОШИБКА: Файл датасета не найден по пути: {file_path}")
        return None
    except (json.JSONDecodeError, TypeError) as e:
        print(f"ОШИБКА: Не удалось прочитать JSON или декодировать Base64. Ошибка: {e}")
        return None

def levenshtein_distance(s1, s2):
    if len(s1) > len(s2): s1, s2 = s2, s1
    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2: distances_.append(distances[i1])
            else: distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

def vqa_evaluation(predict, answers):
    score = 0
    answers_list = answers if isinstance(answers, list) else [answers]
    predict_processed = str(predict).lower().strip().replace("\n", " ")
    for ans in answers_list:
        try: answer_processed = str(ans).lower().strip().replace("\n", " ")
        except: continue
        current_score = 0
        if len(answer_processed.split()) < 5:
            if answer_processed in predict_processed: current_score = 1
        else:
            dist = levenshtein_distance(predict_processed, answer_processed)
            length = max(len(predict_processed), len(answer_processed))
            anls_value = 0.0 if length == 0 else 1.0 - (float(dist) / float(length))
            if anls_value >= 0.5: current_score = anls_value
        if current_score > score: score = current_score
    return score

def evaluate_model(model, processor, dataset, num_samples):
    scores_by_source = {'Overall': []}
    results_data = []
    device = "cuda:0"

    for i, sample in enumerate(dataset):
        if i >= num_samples: break
        
        print(f"\n--- Обработка примера {i+1}/{num_samples} ---")
        
        try:
            content = []
            images_bytes = []
            
            if 'inputs' in sample:
                question = sample['inputs']['question']
                ground_truth = sample['outputs']
                images_bytes.append(sample['inputs']['image']['bytes'])
                source = sample.get('meta', {}).get('source', 'ruCLEVR_unknown')
            else:
                question = sample['question']
                ground_truth = sample['answers']
                images_bytes.extend(sample.get('images', []))
                source = sample.get('source', 'custom_unknown')

            for img_bytes in images_bytes:
                image = Image.open(BytesIO(img_bytes)).convert('RGB')
                content.append({"type": "image", "image": image})

            content.append({"type": "text", "text": question})
            messages = [{"role": "user", "content": content}]

            inputs = processor.apply_chat_template(
                messages, tokenize=True, add_generation_prompt=True,
                return_dict=True, return_tensors="pt"
            ).to(device)

            with torch.no_grad():
                generated_ids = model.generate(**inputs, max_new_tokens=512)
            
            input_len = inputs.input_ids.shape[1]
            output_text = processor.batch_decode(
                generated_ids[:, input_len:], skip_special_tokens=True, clean_up_tokenization_spaces=False
            )
            
            prediction = output_text[0].strip() if output_text else ""
            score = vqa_evaluation(prediction, ground_truth)
            
            if source not in scores_by_source:
                scores_by_source[source] = []
            scores_by_source[source].append(score)
            scores_by_source['Overall'].append(score)
            
            print(f"Источник: {source}")
            print(f"Вопрос: {question}")
            print(f"Истинный ответ: {ground_truth}")
            print(f"Предсказание: {prediction}")
            print(f"Score: {score:.4f}")

            results_data.append({
                'id': i, 'question': question, 'ground_truth': ground_truth,
                'prediction': prediction, 'score': score, 'source': source
            })
            gc.collect()
            torch.cuda.empty_cache()
            
        except Exception as e:
            print(f"Ошибка в примере {i}: {e}")
            scores_by_source.setdefault('Overall', []).append(0)
            if 'source' in locals(): scores_by_source.setdefault(source, []).append(0)
            results_data.append({'id': i, 'error': str(e), 'score': 0})
            
    return scores_by_source, results_data

In [10]:
full_test_dataset = None
dataset_name = ""

if USE_CUSTOM_DATASET:
    dataset_name = "Custom JSONL"
    full_test_dataset = load_custom_jsonl_dataset(CUSTOM_DATASET_PATH)
else:
    dataset_name = "MERA-evaluation/ruCLEVR"
    print("Загрузка датасета ruCLEVR...")
    hf_dataset = datasets.load_dataset(dataset_name, "default")
    full_test_dataset = hf_dataset['test']
    print(f"Датасет загружен: {len(full_test_dataset)} примеров в тестовой выборке.")

if full_test_dataset:
    if NUM_SAMPLES_TO_EVALUATE <= 0 or NUM_SAMPLES_TO_EVALUATE >= len(full_test_dataset):
        dataset_to_run = full_test_dataset
        num_to_run = len(full_test_dataset)
        print("Опция 'оценить весь датасет' выбрана.")
    else:
        dataset_to_run = create_stratified_subset(full_test_dataset, NUM_SAMPLES_TO_EVALUATE)
        num_to_run = len(dataset_to_run)

    print(f"\nЗапуск оценки на {num_to_run} примерах из '{dataset_name}'...")

    final_scores_by_source, final_results = evaluate_model(model, tokenizer, dataset_to_run, num_to_run)

    overall_scores = final_scores_by_source.get('Overall', [])
    if overall_scores:
        avg_score = sum(overall_scores) / len(overall_scores)
        accuracy_05 = sum(1 for s in overall_scores if s >= 0.5) / len(overall_scores)

        print("\n--- ОБЩИЕ РЕЗУЛЬТАТЫ ---")
        print(f"Средний VQA Score: {avg_score:.4f}")
        print(f"Accuracy (score >= 0.5): {accuracy_05:.4f}")
        print(f"Всего оценено примеров: {len(overall_scores)}")

        metrics_by_source = {}
        print("\n--- РЕЗУЛЬТАТЫ ПО ИСТОЧНИКАМ (SOURCE) ---")
        for source, scores in sorted(final_scores_by_source.items()):
            if source == 'Overall' or not scores: continue
            avg_s = sum(scores) / len(scores)
            acc_05 = sum(1 for s in scores if s >= 0.5) / len(scores)
            metrics_by_source[source] = {'average_score': avg_s, 'accuracy_05': acc_05, 'count': len(scores)}
            print(f"\nИсточник: {source}")
            print(f"  Средний VQA Score: {avg_s:.4f}")
            print(f"  Accuracy (>= 0.5): {acc_05:.4f} ({sum(1 for s in scores if s >= 0.5)}/{len(scores)})")

        with open(RESULTS_FILENAME, 'w', encoding='utf-8') as f:
            json.dump({
                'model': "Qwen2.5-VL-32B_sft", 'dataset': dataset_name, 'num_samples': len(overall_scores),
                'overall_metrics': {'average_score': avg_score, 'accuracy_05': accuracy_05},
                'metrics_by_source': metrics_by_source,
                'results': final_results
            }, f, ensure_ascii=False, indent=2)
        print(f"\nПодробные результаты сохранены в файл: {RESULTS_FILENAME}")
else:
    print("Оценка не была запущена, так как датасет не был загружен.")

Загрузка кастомного датасета из /home/jupyter/datasphere/project/data/test.jsonl...
Датасет загружен: 3647 примеров.
  - ruCLEVR: 174 примеров (4.77%), будет взято ~5
  - MWS-Vision-Bench: 196 примеров (5.37%), будет взято ~5
  - ruVQA: 440 примеров (12.06%), будет взято ~12
  - MMBench-ru: 587 примеров (16.10%), будет взято ~16
  - Docmatix: 2250 примеров (61.69%), будет взято ~62

Итоговый размер подвыборки: 100

Запуск оценки на 100 примерах из 'Custom JSONL'...

--- Обработка примера 1/100 ---
Источник: ruVQA
Вопрос: На улице ночь?
Истинный ответ: Нет
Предсказание: Изображение не предоставляет достаточной информации о том, насколько светло или темно на улице. Однако можно сделать следующие наблюдения:

1. **Освещение в помещении**: В комнате включены лампы, что может указывать на то, что на улице уже темно.
2. **Окно**: На фото видна часть окна, но из-за наличия света в комнате и отсутствия видимости через окно сложно определить, какова освещенность снаружи.

Таким образом, **вероя

In [11]:
print(1)

/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:232: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  language = self._parent_header['metadata'].get('language')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:237: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  livy = self._parent_header['metadata'].get('livy')


1


/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:280: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  header = self._parent_header['header']
